In [13]:
import pandas as pd

#### Import do dataset

In [2]:
df_estoque = pd.read_csv('estoque.csv')

In [3]:
df_estoque['CLUSTER'] = ''
df_estoque['PORTE'] = ''

#### Tratamento dos dados

Trata NA para 0 nos campos de valor

In [4]:
df_estoque['VENDA'] = df_estoque['VENDA'].fillna(0) 
df_estoque['VENDA_QTD'] = df_estoque['VENDA_QTD'].fillna(0) 
df_estoque['AVG_ESTOQUE_VLR'] = df_estoque['AVG_ESTOQUE_VLR'].fillna(0) 
df_estoque['AVG_ESTOQUE_QTD'] = df_estoque['AVG_ESTOQUE_QTD'].fillna(0) 

Cria um dataframe com as medianas das colunas de valores para substituir os valores negativos pela mediana do cluster

In [5]:
colunas_interesse = ['AVG_ESTOQUE_VLR', 'AVG_ESTOQUE_QTD', 'VENDA_QTD', 'VENDA']
mediana_df = df_estoque.groupby(['CLUSTER'])[colunas_interesse].quantile(0.5).reset_index()

Sempre que o valor da coluna for negativo, ele será substituido pela mediana do cluster, o objetivo é não ter valores negativos na base de dados.

In [6]:
for col in colunas_interesse:
    mask = df_estoque[col] < 0
    df_estoque.loc[mask, col] = df_estoque.loc[mask, 'CLUSTER'].map(mediana_df.set_index('CLUSTER')[col])


### Calculo da venda média diária

Uma das metas necessárias é a venda média diária, por isso ela é calculada a partir da venda realizada dos ultimos 90 dias

Uma premissa assumida é que para os meses de janeiro e fevereiro foi considerado a mesma venda média de março onde já se tem 90 dias para cálculo.

In [18]:
# Ordenando o DataFrame pela coluna 'DATA'

df = df_estoque.sort_values(['ESTRUTURA_OPER4', 'COD_ESTRUTURA_MERC2', 'ESTRUTURA_OPER0', 'DATA'])
df['DATA'] = pd.to_datetime(df['DATA'], format='%Y-%m-%d')
df['MES_DIA'] = df['DATA'].dt.strftime('%m-%d')

# Calculando a soma dos valores de venda e venda_qtd nos últimos 3 meses para cada grupo
df['SOMA_VENDA_3_MESES'] = df.groupby(['ESTRUTURA_OPER4', 'COD_ESTRUTURA_MERC2', 'ESTRUTURA_OPER0'])['VENDA'].transform(lambda x: x.rolling(window=3, min_periods=1).sum())
df['SOMA_VENDA_QTD_3_MESES'] = df.groupby(['ESTRUTURA_OPER4', 'COD_ESTRUTURA_MERC2', 'ESTRUTURA_OPER0'])['VENDA_QTD'].transform(lambda x: x.rolling(window=3, min_periods=1).sum())

# Obtendo os valores correspondentes à data '2022-03-01' para cada grupo
values_marco = df.loc[df['MES_DIA'] == '03-01', ['ESTRUTURA_OPER4', 'COD_ESTRUTURA_MERC2', 'ESTRUTURA_OPER0', 'SOMA_VENDA_3_MESES', 'SOMA_VENDA_QTD_3_MESES']]

# Substituindo os valores de 'SOMA_VENDA_3_MESES' e 'SOMA_VENDA_QTD_3_MESES' nas datas de janeiro e feveiro pelos valores de março
df['SOMA_VENDA_3_MESES'] = df.apply(
    lambda row: values_marco[(values_marco['ESTRUTURA_OPER4'] == row['ESTRUTURA_OPER4']) & 
                                  (values_marco['COD_ESTRUTURA_MERC2'] == row['COD_ESTRUTURA_MERC2']) & 
                                  (values_marco['ESTRUTURA_OPER0'] == row['ESTRUTURA_OPER0'])]['SOMA_VENDA_3_MESES'].values[0]
    if row['MES_DIA'] in ['01-01', '02-01'] else row['SOMA_VENDA_3_MESES'],
    axis=1
)

df['SOMA_VENDA_QTD_3_MESES'] = df.apply(
    lambda row: values_marco[(values_marco['ESTRUTURA_OPER4'] == row['ESTRUTURA_OPER4']) & 
                                  (values_marco['COD_ESTRUTURA_MERC2'] == row['COD_ESTRUTURA_MERC2']) & 
                                  (values_marco['ESTRUTURA_OPER0'] == row['ESTRUTURA_OPER0'])]['SOMA_VENDA_QTD_3_MESES'].values[0]
    if row['MES_DIA'] in ['01-01', '02-01'] else row['SOMA_VENDA_QTD_3_MESES'],
    axis=1
)

# Calculando a média diária considerando a soma dos valores de venda e venda_qtd nos últimos 3 meses
df['AVG_VND_DIARIA_VLR'] = df['SOMA_VENDA_3_MESES'] / 90
df['AVG_VND_DIARIA_QTD'] = df['SOMA_VENDA_QTD_3_MESES'] / 90

df = df.drop(['SOMA_VENDA_3_MESES', 'SOMA_VENDA_QTD_3_MESES', 'MES_DIA'], axis=1, errors='ignore')
df_estoque = df

#### Calculo das metas para os cenários Conservador, Realista e Agressivo

O cenário conservador é o valor do segundo quartil. <br>
O cenário realista é o valor do terceiro quartil.<br>
O cenário agressivo é o valor do valor máximo do cluster.

In [ ]:
# Define os quartis desejados
quantis = [0.50, 0.75, 1]

colunas_interesse = ['AVG_ESTOQUE_VLR', 'AVG_ESTOQUE_QTD', 'AVG_VND_DIARIA_VLR','AVG_VND_DIARIA_QTD']

# Agrupa o DataFrame pelos grupos desejados e calcule os quantis
quartis_df = df_estoque.groupby(['CLUSTER', 'DATA'])[colunas_interesse].quantile(quantis).unstack()

In [ ]:
# Renomeia as colunas para indicar o quartil
quartis_df.columns = [f'{col[0]}_q{int(col[1]*100)}' for col in quartis_df.columns]

In [ ]:
quartis_df

AVG_ESTOQUE_VLR_q50  AVG_ESTOQUE_VLR_q75  \
CLUSTER DATA                                                   
        2022-01-01            41.678687           135.033860   
        2022-02-01            41.678687           127.815898   
        2022-03-01            41.678687           123.388093   
        2022-04-01            41.678687           118.350038   
        2022-05-01            41.678687           120.477174   
        2022-06-01            41.678687           117.373199   
        2022-07-01            41.678687           114.405000   
        2022-08-01            41.678687           107.012748   
        2022-09-01            49.858643           160.939329   
        2022-10-01            44.282066           153.464271   
        2022-11-01            56.428786           190.150285   
        2022-12-01            98.225750           269.929825   

                    AVG_ESTOQUE_VLR_q100  AVG_ESTOQUE_QTD_q50  \
CLUSTER DATA                                                    
        2022-01-01         133260.330455             4.934750   
        2022-02-01          63478.015909             4.934750   
        2022-03-01         121045.398696             4.934750   
        2022-04-01          70412.670000             4.934750   
        2022-05-01         128911.673333             4.934750   
        2022-06-01          66411.485417             4.934750   
        2022-07-01         132871.716250             4.934750   
        2022-08-01          56143.190000             4.934750   
        2022-09-01          90231.147055             5.818711   
        2022-10-01         116139.774667             5.000000   
        2022-11-01         157049.421237             6.494681   
        2022-12-01         198987.975529            12.513797   

                    AVG_ESTOQUE_QTD_q75  AVG_ESTOQUE_QTD_q100  \
CLUSTER DATA                                                    
        2022-01-01            16.612028           6696.000000   
        2022-02-01            15.537226           6696.000000   
        2022-03-01            14.670483           8370.434783   
        2022-04-01            13.860837           6696.000000   
        2022-05-01            14.431472          11126.200000   
        2022-06-01            13.908034           6606.000000   
        2022-07-01            13.792535          14100.833333   
        2022-08-01            12.764706           6606.000000   
        2022-09-01            22.610034           5137.130631   
        2022-10-01            21.255319           8471.013333   
        2022-11-01            27.911409          18477.969072   
        2022-12-01            39.285714          26666.823529   

                    AVG_VND_DIARIA_VLR_q50  AVG_VND_DIARIA_VLR_q75  \
CLUSTER DATA                                                         
        2022-01-01               11.602389              187.766583   
        2022-02-01               11.356556              186.779306   
        2022-03-01               11.243833              186.043444   
        2022-04-01               11.446833              190.803250   
        2022-05-01               11.500000              195.604889   
        2022-06-01               12.446111              199.365889   
        2022-07-01               12.762278              204.869417   
        2022-08-01               12.500556              201.648500   
        2022-09-01               13.631889              209.630444   
        2022-10-01               12.930000              212.858000   
        2022-11-01               12.394889              217.513806   
        2022-12-01               35.786222              341.741000   

                    AVG_VND_DIARIA_VLR_q100  AVG_VND_DIARIA_QTD_q50  \
CLUSTER DATA                                                          
        2022-01-01            142146.114778                0.755556   
        2022-02-01            142146.114778                0.744444   
        2022-03-01            142146

In [ ]:
# Reset index para tornar os grupos como colunas novamente
quartis_df = quartis_df.reset_index()

# Junta os quartis de volta ao DataFrame original
df_estoque = pd.merge(df_estoque, quartis_df, on=[ 'CLUSTER', 'DATA'])

Cria um dataframe com valores minimos para as colunas de interesse, isso é usado para garantir que as metas não serão negativas

In [ ]:
# Agrupa o DataFrame pelos grupos desejados e encontra o valor maximo
min = df_estoque[(df_estoque['AVG_ESTOQUE_VLR']>=0) & (df_estoque['AVG_ESTOQUE_QTD']>=0) & (df_estoque['AVG_VND_DIARIA_VLR']>=0) & (df_estoque['AVG_VND_DIARIA_QTD']>=0)]
min= min.groupby(['CLUSTER', 'DATA',])[colunas_interesse].min()

# Renomeia as colunas para indicar o quartil
min.columns = [f'{col}_MIN' for col in min.columns]

# Reset index para tornar os grupos como colunas novamente
min = min.reset_index()

In [ ]:
# Junta os minimos  ao DataFrame original
df_estoque = pd.merge(df_estoque, min, on=[ 'CLUSTER','DATA'])

#### Cria as colunas de metas

É feita uma análise das metas sugeridas, caso o valor realizado seja menor que a meta sugerida é mantido o valor realizado como meta

In [ ]:
#Cria as colunas de metas se a meta sugerida for menor que zero utiliza o mínimo do cluster
df_estoque['META_ESTOQUE_VLR_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_VLR_MIN'] if x['AVG_ESTOQUE_VLR_q100']< 0 else x['AVG_ESTOQUE_VLR_q100'], axis=1)
df_estoque['META_ESTOQUE_QTD_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_QTD_MIN'] if x['AVG_ESTOQUE_QTD_q100']< 0 else x['AVG_ESTOQUE_QTD_q100'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_VLR_MIN'] if x['AVG_VND_DIARIA_VLR_q100']< 0 else x['AVG_VND_DIARIA_VLR_q100'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_QTD_MIN'] if x['AVG_VND_DIARIA_QTD_q100']< 0 else x['AVG_VND_DIARIA_QTD_q100'], axis=1)

In [ ]:
#Cria as colunas de metas se a meta sugerida for menor que zero utiliza o mínimo do cluster
df_estoque['META_ESTOQUE_VLR_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_VLR'] if (x['META_ESTOQUE_VLR_CONSERVADOR']> x['AVG_ESTOQUE_VLR']) and (x['AVG_ESTOQUE_VLR'] >= 0) else x['META_ESTOQUE_VLR_CONSERVADOR'], axis=1)
df_estoque['META_ESTOQUE_QTD_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_QTD'] if (x['META_ESTOQUE_QTD_CONSERVADOR']> x['AVG_ESTOQUE_QTD']) and (x['AVG_ESTOQUE_QTD']>= 0) else x['META_ESTOQUE_QTD_CONSERVADOR'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_VLR'] if (x['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_CONSERVADOR']> x['AVG_VND_DIARIA_VLR']) and (x['AVG_VND_DIARIA_VLR']>=0) else x['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_CONSERVADOR'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_CONSERVADOR'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_QTD']  if (x['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_CONSERVADOR']> x['AVG_VND_DIARIA_QTD']) and (x['AVG_VND_DIARIA_QTD']>=0) else x['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_CONSERVADOR'], axis=1)

In [ ]:
#Cria as colunas de metas se a meta sugerida for menor que zero utiliza o mínimo do cluster
df_estoque['META_ESTOQUE_VLR_REALISTA'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_VLR_MIN'] if x['AVG_ESTOQUE_VLR_q75']< 0 else x['AVG_ESTOQUE_VLR_q75'], axis=1)
df_estoque['META_ESTOQUE_QTD_REALISTA'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_QTD_MIN'] if x['AVG_ESTOQUE_QTD_q75']< 0 else x['AVG_ESTOQUE_QTD_q75'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_REALISTA'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_VLR_MIN'] if x['AVG_VND_DIARIA_VLR_q75']< 0 else x['AVG_VND_DIARIA_VLR_q75'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_REALISTA'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_QTD_MIN'] if x['AVG_VND_DIARIA_QTD_q75']< 0 else x['AVG_VND_DIARIA_QTD_q75'], axis=1)

In [ ]:
#Cria as colunas de metas se a meta sugerida for menor que zero utiliza o mínimo do cluster
df_estoque['META_ESTOQUE_VLR_REALISTA'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_VLR'] if (x['META_ESTOQUE_VLR_REALISTA']> x['AVG_ESTOQUE_VLR']) and (x['AVG_ESTOQUE_VLR'] >= 0) else x['META_ESTOQUE_VLR_REALISTA'], axis=1)
df_estoque['META_ESTOQUE_QTD_REALISTA'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_QTD'] if (x['META_ESTOQUE_QTD_REALISTA']> x['AVG_ESTOQUE_QTD']) and (x['AVG_ESTOQUE_QTD']>= 0) else x['META_ESTOQUE_QTD_REALISTA'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_REALISTA'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_VLR'] if (x['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_REALISTA']> x['AVG_VND_DIARIA_VLR']) and (x['AVG_VND_DIARIA_VLR']>=0) else x['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_REALISTA'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_REALISTA'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_QTD']  if (x['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_REALISTA']> x['AVG_VND_DIARIA_QTD']) and (x['AVG_VND_DIARIA_QTD']>=0) else x['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_REALISTA'], axis=1)

In [ ]:
#Cria as colunas de metas se a meta sugerida for menor que zero utiliza o mínimo do cluster
df_estoque['META_ESTOQUE_VLR_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_VLR_MIN'] if x['AVG_ESTOQUE_VLR_q50']< 0 else x['AVG_ESTOQUE_VLR_q50'], axis=1)
df_estoque['META_ESTOQUE_QTD_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_QTD_MIN'] if x['AVG_ESTOQUE_QTD_q50']< 0 else x['AVG_ESTOQUE_QTD_q50'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_VLR_MIN'] if x['AVG_VND_DIARIA_VLR_q50']< 0 else x['AVG_VND_DIARIA_VLR_q50'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_QTD_MIN'] if x['AVG_VND_DIARIA_QTD_q50']< 0 else x['AVG_VND_DIARIA_QTD_q50'], axis=1)

In [ ]:
#Cria as colunas de metas se a meta sugerida for menor que zero utiliza o mínimo do cluster
df_estoque['META_ESTOQUE_VLR_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_VLR'] if (x['META_ESTOQUE_VLR_AGRESSIVO']> x['AVG_ESTOQUE_VLR']) and (x['AVG_ESTOQUE_VLR'] >= 0) else x['META_ESTOQUE_VLR_AGRESSIVO'], axis=1)
df_estoque['META_ESTOQUE_QTD_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_ESTOQUE_QTD'] if (x['META_ESTOQUE_QTD_AGRESSIVO']> x['AVG_ESTOQUE_QTD']) and (x['AVG_ESTOQUE_QTD']>= 0) else x['META_ESTOQUE_QTD_AGRESSIVO'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_VLR'] if (x['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_AGRESSIVO']> x['AVG_VND_DIARIA_VLR']) and (x['AVG_VND_DIARIA_VLR']>=0) else x['META_ESTOQUE_VND_MEDIA_DIARIA_VLR_AGRESSIVO'], axis=1)
df_estoque['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_AGRESSIVO'] = df_estoque.apply(lambda x: x['AVG_VND_DIARIA_QTD']  if (x['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_AGRESSIVO']> x['AVG_VND_DIARIA_QTD']) and (x['AVG_VND_DIARIA_QTD']>=0) else x['META_ESTOQUE_VND_MEDIA_DIARIA_QTD_AGRESSIVO'], axis=1)

Selecionando somente as colunas necessárias para o dataframe final

In [ ]:
#Cria um dataframe apenas com as colunas necessarias
df_estoque = df_estoque[['ESTRUTURA_OPER0', 'ESTRUTURA_OPER1', 'ESTRUTURA_OPER1','PORTE','COD_ESTRUTURA_OPER4','ESTRUTURA_OPER4', 'COD_ESTRUTURA_MERC2','DATA','AVG_ESTOQUE_VLR', 'AVG_ESTOQUE_QTD', 'AVG_VND_DIARIA_VLR', 'AVG_VND_DIARIA_QTD', 'META_ESTOQUE_VLR_CONSERVADOR','META_ESTOQUE_QTD_CONSERVADOR','META_ESTOQUE_VND_MEDIA_DIARIA_VLR_CONSERVADOR', 'META_ESTOQUE_VND_MEDIA_DIARIA_QTD_CONSERVADOR', 'META_ESTOQUE_VLR_REALISTA','META_ESTOQUE_QTD_REALISTA','META_ESTOQUE_VND_MEDIA_DIARIA_VLR_REALISTA', 'META_ESTOQUE_VND_MEDIA_DIARIA_QTD_REALISTA', 'META_ESTOQUE_VLR_AGRESSIVO','META_ESTOQUE_QTD_AGRESSIVO','META_ESTOQUE_VND_MEDIA_DIARIA_VLR_AGRESSIVO', 'META_ESTOQUE_VND_MEDIA_DIARIA_QTD_AGRESSIVO'] ]

In [ ]:
#Adicionar 1 ano a data atual
df_estoque['DATA'] = pd.to_datetime(df_estoque['DATA'], format='%Y-%m-%d')
df_estoque['DATA'] = df_estoque['DATA'].apply(lambda x: x.replace(year=2023))

In [ ]:
df_estoque.to_csv('metas_estoque.csv')